# Setup or Reset

## Warning!!!!
### Running this code will delete an existing database as specified in your creds.json file.

We will start with a wrapper around Snowpark to simplify connections and the ability to choose different compute instances.  We need a way to save state through out the project too.

In [ ]:
state_dict = {
    "connection_parameters": {"user": "john",  
                              "database": "CITIBIKEML_JOHN",  
                              "schema": "DEMO",  
                              "password": "",
                              "account": "an06783.eu-central-1",
                              "role": "DBA_CITIBIKE",  
    },
    "compute_parameters" : {
              "default_warehouse": "XSMALL_WH",  
              "task_warehouse": "XSMALL_WH",  
              "load_warehouse": "LARGE_WH",  
              "fe_warehouse": "XXLARGE_WH",  
              "train_warehouse": "XXLARGE_WH"  
    },
    "download_base_url":"https://s3.amazonaws.com/tripdata/",
    "load_table_name":"RAW_",
    "trips_table_name":"TRIPS",
    "load_stage_name":"LOAD_STAGE",
    "model_stage_name":"MODEL_STAGE",
    "model_id": "",
    "key_pair_name":"", 
    "public_key":""
}

In [ ]:
import json
with open('./include/state.json', 'w') as sdf:
    json.dump(state_dict, sdf)

We will connect with username and password initially to create a ssh keypair which we will use going forward.

In [ ]:
import snowflake.snowpark as snp
import os, json, uuid
import getpass

with open('./include/state.json') as sdf:
    state_dict = json.load(sdf)    
state_dict['model_id'] = str(uuid.uuid1()).replace('-', '_')
state_dict['connection_parameters']['password'] = getpass.getpass()

session = snp.Session.builder.configs(state_dict["connection_parameters"]).create()

If you do not currently have these users and roles created you can use something like the following to create them.

In [ ]:
sql_cmd='''use role securityadmin;

    -- create role
    create role dba_citibike;
    grant role sysadmin to role dba_citibike;

    use role securityadmin;

    -- create user
    CREATE USER john PASSWORD = 'snow2022' LOGIN_NAME = 'john' DEFAULT_ROLE = "DBA_CITIBIKE" MUST_CHANGE_PASSWORD = TRUE;
    GRANT ROLE "DBA_CITIBIKE" TO USER john;

    use role sysadmin;'''
#session.sql(sql_cmd).collect()

Likewise for compute warehouses the following code example can create them automatically

In [ ]:
# for wh in state_dict['compute_parameters'].values():
#     print("Creating warehouse "+wh)
#     session.sql("CREATE WAREHOUSE IF NOT EXISTS "+wh+\
#             " WITH WAREHOUSE_SIZE = '"+wh.split('_')[0]+\
#             "' WAREHOUSE_TYPE = 'STANDARD' AUTO_SUSPEND = 300 AUTO_RESUME = TRUE initially_suspended = true;")\
#         .collect()
    
#     #session.sql("GRANT OWNERSHIP ON WAREHOUSE "+wh+" TO ROLE DBA_CITBIKE").collect() 



Create the database and schema for this project.

In [ ]:
_ = session.sql('CREATE OR REPLACE DATABASE '+state_dict['connection_parameters']['database']).collect()
_ = session.sql('CREATE SCHEMA '+state_dict['connection_parameters']['schema']).collect() 

Quick test of changing compute types

In [ ]:
session.get_current_warehouse()

In [ ]:
session.use_warehouse(state_dict['compute_parameters']['load_warehouse'])

In [ ]:
session.get_current_warehouse()

In [ ]:
session.close()

Save the updated state dictionary for project team use.

In [ ]:
import json
with open('./include/state.json', 'w') as sdf:
    json.dump(state_dict, sdf)

 We add a python function to simplify starting a session.  We will add a new dictionary of compute resources available to our DE and ML Ops pipeline. 

In [ ]:
%%writefile dags/snowpark_connection.py

def snowpark_connect(state_file='./include/state.json'):
    import snowflake.snowpark as snp
    import os, json, uuid
    
    with open(state_file) as sdf:
        state_dict = json.load(sdf)    
    state_dict["model_id"] = str(uuid.uuid1()).replace('-', '_')
    
    session=None
    session = snp.Session.builder.configs(state_dict["connection_parameters"]).create()
    session.use_warehouse(state_dict['compute_parameters']['default_warehouse'])
    return session, state_dict

Test the function that users will use.

In [ ]:
from dags.snowpark_connection import snowpark_connect
session, state_dict = snowpark_connect()

In [ ]:
session.close()